In [4]:
import oommfc as oc
import discretisedfield as df
import numpy as np
import ipyvolume as iv
import matplotlib.cm as cm
%matplotlib inline

In [5]:
def quiver(field, Ms, coord='z', **kwargs):
    x, y, z = np.array(list(field.mesh.coordinates)).T
    u, v, w = field.array.T
    u = u.flatten()
    v = v.flatten()
    w = w.flatten()
    iv.figure(width=600, height=800)
    iv.xlim(-diametre/2, diametre/2)
    iv.ylim(-diametre/2, diametre/2)
    iv.zlim(0, thickness)
    
    if coord == 'x':
        p = u.copy()
    elif coord == 'y':
        p = v.copy()
    elif coord == 'z':
        p = w.copy()
    
    c = p / Ms * 0.5 + 0.5
    # cm.{colormap name} to get a specific colourmap.
    # Not sure how to change it such that we can fetch
    # it to pass as an option.
    colors = cm.RdBu(c)
    return iv.quiver(x, y, z, u, v, w, color=colors[:, :3], **kwargs)

In [6]:
diametre = 100e-9
thickness = 10e-9
d = 2e-9
Ms = 8.84e5

mesh = oc.Mesh(p1=(-diametre/2, -diametre/2, 0), p2=(diametre/2, diametre/2, thickness), cell=(d, d, d))

system = oc.System(name="skyrmion")

def Ms_fun(pos):
    x, y, z = pos
    if x**2 + y**2 <= (diametre/2)**2:
        return Ms
    else:
        return 0
    
def m_init(pos):
    x, y, z = pos
    if x**2 + y**2 <= (20e-9)**2:
        return (0, 0, -1)
    else:
        return (0, 0, 1)

system.hamiltonian = oc.Exchange(A=8.78e-12) + oc.DMI(D=1.58e-3, crystalclass="t") + oc.Demag() + oc.Zeeman(H=(0, 0, 2e5))
system.m = df.Field(mesh, value=m_init, norm=Ms_fun)

md = oc.MinDriver()
md.drive(system)


fig = quiver(system.m, Ms, coord='z', size=3)
iv.gcc()

2018/6/22 15:43: Calling OOMMF (skyrmion/skyrmion.mif) ... [4.7s]


In [7]:
diametre = 50e-9
thickness = 10e-9
d = 1e-9

mesh = oc.Mesh(p1=(-diametre/2, -diametre/2, 0), p2=(diametre/2, diametre/2, thickness), pbc='xy', cell=(d, d, d))

system = oc.System(name="skyrmion_pbc")

system.hamiltonian = oc.Exchange(A=8.78e-12) + oc.DMI(D=4e-3, crystalclass="t") + oc.Zeeman(H=(0, 0, 1e5))
system.m = df.Field(mesh, value=m_init, norm=8e6)

md = oc.MinDriver()
md.drive(system)

fig = quiver(system.m, Ms, coord='z', size=3)
iv.gcc()

2018/6/22 15:43: Calling OOMMF (skyrmion_pbc/skyrmion_pbc.mif) ... [2.9s]
